In [1]:
import sys
from pathlib import Path
PROJECT_DIR = Path.cwd()
if PROJECT_DIR.stem == 'notebooks':
    PROJECT_DIR = PROJECT_DIR.parent
    sys.path.insert(0, '..')
    %load_ext autoreload
    %autoreload 2

In [2]:
from src.experiments.thermoino import Thermoino, list_com_ports
from src.experiments.stimuli_function import StimuliFunction

In [3]:
import numpy as np
minimal_desired_duration = 200 # in seconds
periods = [67, 20] # [0] is the baseline and [1] the modulation; in seconds
frequencies = 1./np.array(periods)
amplitudes = [1, 1.5] # temp range is 2 * sum(amplitudes): max @ VAS 70, min @ VAS 0
sample_rate = 60
seed = 463 # use None for random seed
baseline_temp = 39.2 # @ VAS 35

stimuli = StimuliFunction(
    minimal_desired_duration,
    frequencies,
    amplitudes,
    sample_rate,
    random_periods=True,
    seed=seed)
stimuli.add_baseline_temp(baseline_temp)
stimuli.add_plateaus(plateau_duration=20, n_plateaus=4, add_at_start="random", add_at_end=True)


In [4]:
list_com_ports()

COM3: Serielles USB-Gerät (COM3) [USB VID:PID=2A03:0042 SER=85531303231351519040 LOCATION=1-4]


In [6]:
import time
# list_com_ports() # list all available serial ports
port = "COM3" # e.g. "COM12"
luigi = Thermoino(
    port=port,
    temp_baseline=32, # has to be the same as in MMS
    rate_of_rise=5) # has to be the same as in MMS

# Use luigi for complex temperature courses:
luigi.connect()
luigi.init_ctc(bin_size_ms=500)
luigi.create_ctc(
    temp_course = stimuli.wave,
    sample_rate = stimuli.sample_rate,
    rate_of_rise_option = "mms_program")
luigi.load_ctc()
luigi.trigger()
luigi.prep_ctc()
luigi.exec_ctc()
luigi.set_temp(32)
luigi.close()

# Use luigi to set temperatures:
luigi.connect()
luigi.trigger()
luigi.set_temp(42)
time.sleep(4)
luigi.set_temp(32)
luigi.close()


INFO: Connected to Thermoino @ COM3
INFO: Sent 'INITCTC' (.init_ctc) to Arduino, received output: 1
INFO: Loading ctc into Arduino ...
INFO: Finished loading ctc into Arduino.
INFO: Sent 'START' (.trigger) to Arduino, received output: 1
INFO: Preparing ctc for execution. Set starting temperature and wait 1.9 s for the temperature to be reached.
INFO: Sent 'MOVE' (.set_temp) to Arduino with temperature 39.2°C, received output: 1
INFO: Temperature is 39.2°C. The ctc is ready to be executed.
INFO: Sent 'EXECCTC' (.exec_ctc) to Arduino, received output: 1
INFO: Set temperature to 39.1°C after the ctc was executed.
INFO: Sent 'MOVE' (.set_temp) to Arduino with temperature 32°C, received output: 1
INFO: Closed connection to Thermoino @ COM3
INFO: Connected to Thermoino @ COM3
INFO: Sent 'START' (.trigger) to Arduino, received output: -10
INFO: Sent 'MOVE' (.set_temp) to Arduino with temperature 42°C, received output: 1
INFO: Sent 'MOVE' (.set_temp) to Arduino with temperature 32°C, received 

In [91]:
import numpy as np
periods = [67, 20] # [0] is the baseline and [1] the modulation; in seconds
factor = 0.5
n = 100
noise = np.random.uniform(
        -factor * periods[1], 
        factor * periods[1],
        size = n)
noise -= noise.mean()
np.random.shuffle(noise)
a = np.round(noise)
sum(a)

2.0

In [143]:
2 // 1


2

In [149]:
a = [1, 2, 3]
b = [4, 5]
a.append(b)
a = np.array(a)
a.ravel()

C:\Users\Leopold\AppData\Local\Temp\ipykernel_17772\4106090207.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  a = np.array(a)


array([1, 2, 3, list([4, 5])], dtype=object)

In [151]:
a = np.array([1, 2, 3])
b = np.array([4, 5])
a[1]+b

array([6, 7])

In [164]:
(np.arange(5) % 2) #% 2 == 1

False

In [160]:
a.any

<function ndarray.any>

In [81]:
noise = np.random.uniform(
    -factor * periods[1], 
    factor * periods[1],
    size = int(n/2))
# double the noise with inverted values to sum to 0
noise = np.concatenate((noise, -noise))
# add 0 if length of n is odd
if n % 2 == 1:
    noise = np.append(noise, 0)
np.random.shuffle(noise)
b = np.round(noise)
sum(b)

0.0

In [129]:
# create noise
noise = np.random.randint(
        -factor * periods[1], 
        factor * periods[1] + 1,
        size = n)
# adjust the noise so it sums to 0
adj = -int(noise.sum() / n)
noise[:n // 2] += adj
noise[n // 2:] -= adj
np.random.shuffle(noise)

102

In [137]:
int(13/2) == 13//2

True

In [165]:

def psychopy_to_pixel_coordinates(pos, win_size, win_pos):
    """Converts PsychoPy coordinates to pixel coordinates"""
    x, y = pos
    width, height = win_size
    win_x, win_y = win_pos
    pixel_x = (x + 1) / 2 * width + win_x
    pixel_y = (1 - y) / 2 * height + win_y
    return pixel_x, pixel_y


In [166]:
psychopy_to_pixel_coordinates((0, ), (1920, 1080), (0, 0))

(960.0, 540.0)

In [201]:
import mouse
mouse.get_position()[1]
# mouse.move(100, 842, absolute=True, duration=0.2)

1160

In [194]:
from win32api import GetSystemMetrics
import ctypes
# needed to get the correct screen resolution (bc of DPI scaling)
# this method is safe to use with all types of monitors, whether they use DPI scaling or not
user32 = ctypes.windll.user32
user32.SetProcessDPIAware()

width = user32.GetSystemMetrics(0)
height = user32.GetSystemMetrics(1)
# in a multi-monitor setup, you can use the screeninfo library, for instance
# >>> from screeninfo import get_monitors
# >>> for m in get_monitors():
# >>>    print(str(m))
# this prints out the resolution (width and height), position (x, y), and other information for each monitor 
print(width, height)


2256 1504


In [229]:
import mouse

mouse.move(mouse.get_position()[0], 872, absolute=True, duration=0)